https://platform.openai.com/docs/guides/function-calling 

`We recommend that you use no more than 20 tools in a single API call.`

`Developers typically see a reduction in the model's ability to select the correct tool once they have between 10-20 tools defined.`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
from datetime import datetime

def get_delivery_date(order_id: str):
    # here will be really compilacted function logic
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [4]:
get_delivery_date_tool = {
    "type": "function",
    "function": {
        "name": "get_delivery_date",  # it is good to use real function name here
        "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The customer's order ID.",
                },
            },
            "required": ["order_id"],
            "additionalProperties": False,
        },
    }
}

In [5]:
def get_weather(location: str, unit: str):
    # e.g. may call external API
    return "Sunny :) 23 C."

In [6]:
get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "Location to check weather in."},
                "unit": {"type": "string", "enum": ["C", "F"], "description": "Unit of temperature: C - Celsius, F - Fahrenheit"},
            },
            "required": ["location", "unit"],
            "additionalProperties": False,
        },
    },
}

In [7]:
funcs_to_call = {
    "get_weather": get_weather,
    "get_delivery_date": get_delivery_date,
}

In [8]:
tools = [get_weather_tool, get_delivery_date_tool]

messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Hi, can you tell me the delivery date for my order?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [9]:
response.model_dump()

{'id': 'chatcmpl-AYgguQwiD8ftgPqLdrJFNNHxqkvG2',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Could you please provide me with your order ID?',
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': None}}],
 'created': 1732830620,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_0705bf87c0',
 'usage': {'completion_tokens': 11,
  'prompt_tokens': 149,
  'total_tokens': 160,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [10]:
messages.append({"role": "assistant", "content": response.choices[0].message.content})
messages.append({"role": "user", "content": "It is 902834-235-gdf"})

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [11]:
response.model_dump()

{'id': 'chatcmpl-AYggvyh4PVpWTLEWSaeoubl64sWYq',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_Yj4lLFn3DQXb1MqZ0ehs92JA',
      'function': {'arguments': '{"order_id":"902834-235-gdf"}',
       'name': 'get_delivery_date'},
      'type': 'function'}]}}],
 'created': 1732830621,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_0705bf87c0',
 'usage': {'completion_tokens': 21,
  'prompt_tokens': 176,
  'total_tokens': 197,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [12]:
import json

tool_call = response.choices[0].message.tool_calls[0].model_dump()
arguments = json.loads(tool_call['function']['arguments'])
function_name = tool_call['function']['name']

In [13]:
function_name, arguments

('get_delivery_date', {'order_id': '902834-235-gdf'})

In [14]:
result = funcs_to_call[function_name](**arguments)
result

'2024-11-28 22:50:21'

In [15]:
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        **arguments,
        "result": result
    }),
    "tool_call_id": response.model_dump()['choices'][0]['message']['tool_calls'][0]['id']
}
function_call_result_message

{'role': 'tool',
 'content': '{"order_id": "902834-235-gdf", "result": "2024-11-28 22:50:21"}',
 'tool_call_id': 'call_Yj4lLFn3DQXb1MqZ0ehs92JA'}

In [16]:
messages.append(response.choices[0].message)  # func call
messages.append(function_call_result_message)  # function result

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [17]:
response.choices[0].message.role, response.choices[0].message.content

('assistant',
 'Your order is scheduled to be delivered on **November 28, 2024, at 10:50 PM**.')